In [1]:
from google.oauth2 import service_account
import gspread
import pandas as pd
import numpy as np

In [12]:
from pathlib import Path

In [14]:
BASE_DIR = Path.cwd()
IMAGES_DIR = BASE_DIR / 'images'

In [16]:
str(IMAGES_DIR)

'/Users/PutosHackers/Documents/Linterna Verde/Apps/query/images'

In [2]:
SCOPES = ['https://www.googleapis.com/auth/spreadsheets',
         'https://www.googleapis.com/auth/drive',
         'https://www.googleapis.com/auth/drive.file']
SERVICE_ACCOUNT_FILE = 'keys.json'

class Client:
    def __init__(self, sp, sap):
        self.scopes = sp
        self.service_account_file = sap
        
        #initialize the creds and client
        creds = service_account.Credentials.from_service_account_file(self.service_account_file,scopes=self.scopes)
        self.client = gspread.authorize(creds)

In [3]:
class SpreadSheet(Client):
    def __init__(self, sp, sap, spdst_name):
        super().__init__(sp, sap)
        self.spreadsheet = self.client.open_by_url(url)
        
    def dict_to_df(self, name='Diccionario'):
        dict_worksheet = self.spreadsheet.worksheet(name)
        df = pd.DataFrame(dict_worksheet.get_all_records())
        return df
    
    def append_query_to_spreadsheet(query, query_name='Query'):    
        try:
            self.spreadsheet.add_worksheet(title=query_name, rows="100", cols="20")
        except:
            pass
        
        self.spreadsheet.worksheet(query_name).update('A1', query_name)
        self.spreadsheet.worksheet(query_name).update('A4', query)

In [4]:
url = 'https://docs.google.com/spreadsheets/d/1iilHcaQqvJj6ShrnIbRFlKeqjaCnoNxoza11WGuYwps/edit#gid=0'

In [5]:
s = SpreadSheet(SCOPES, SERVICE_ACCOUNT_FILE, url)
df = s.dict_to_df()

In [6]:
df.columns.values

array(['Paro', 'Actores', 'Reforma a la salud', 'Instituciones', 'NOT',
       'Filtro por términos en tweets falsos',
       'Filtro por palabras de desinformación'], dtype=object)

In [7]:
otros = df.columns[1:4]

In [8]:
class Query:
    not_col = False
    
    def __init__(self,df_dict):
        self.dictionary = df_dict
        self.preprocess_dictionary()
    
    #Actions in Dictionary
    
    # Check for a NOT column
    def NOT_col_check(self):
        self.not_col = 'NOT' in self.dictionary.columns
        
    #Repalce blank spaces with NaN
    def replace_blank_with_nan(self):
        self.dictionary.replace("", np.nan, inplace=True) # replace empty string with np.nan        
      
    #Change position of the NOT column
    def NOT_col_last(self):
        NOT = self.dictionary['NOT']
        self.dictionary.drop(columns=['NOT'], inplace=True)
        self.dictionary['NOT'] = NOT
    
    #Preprocess dict
    def preprocess_dictionary(self):
        self.NOT_col_check()
        self.replace_blank_with_nan()
        if self.not_col:
            self.NOT_col_last()
    
    #Actions in Dictionary Temp
    
    # Combine columns given in col_list
    def combine_columns(self, col_list):
            df_combined = self.dictionary
            col_pivot= col_list[0]

            for c in col_list[1:]:
                for val  in df[c]:
                    df_combined = df_combined.append({col_pivot:val}, ignore_index=True)

                df_combined.drop(c,axis=1, inplace=True)

            df_combined.rename(columns={col_pivot:'(' + ' OR '.join(col_list) + ')'},inplace=True)

            self.dictionary = df_combined
        
    # Join values in a column with OR
    def join_col_vals_OR(self):
        df_temp = self.dictionary
        query_list = [] #lista de palabras concatenadas por OR
        num_cols = np.shape(df_temp)[1] #número de columnas en el documento

        for i in range(num_cols):
            col_val = df_temp.iloc[:,i].dropna().values #valores de la columna filtrando NaN

            if len(col_val)==0:
                print("Columna {} : {} --> VACÍA".format(i,df_temp.columns[i]))
                continue
            for j in range(len(col_val)):
                if col_val[j][0] == "\"":
                    continue
                else:
                    col_val[j] = '('+ col_val[j] + ')'

            query_list.append('(' + ' OR '.join(col_val) + ')') #une los valores con un OR y los agrega a la lista query_list

        return query_list
    
    # Join column with AND in a query. 
    #In case NOT colum exist it goes last and is joined with NOT.
    def get_query(self):
        query_list = self.join_col_vals_OR()
        
        if self.not_col:
            query = '(' + ' AND '.join(query_list[:-1]) + ' NOT ' + query_list[-1] + ')'

        else:
            query = '(' + ' AND '.join(query_list[:]) + ')'

        query = query + ' AND metaData.source.socialOriginType:“twitter”' #Filter for twitter only

        return query
    
    #Print the logic expression of the query
    def get_logic_expression(self):
        print(' AND '.join(self.dictionary.columns))

In [9]:
q = Query(df)

In [31]:
q.combine_columns(otros)

In [32]:
q.dictionary.head(15)

,Filtro sobre el paro,(Filtro de actores OR Filtro de reforma a la salud OR Filtro de instituciones),Filtro por términos en tweets falsos,Filtro por palabras de desinformación,NOT
0,paro NEAR nacional,"""fernando ruiz""",desinfroma*,sarmiento NEAR angulo,"""wilson ruiz"""
1,soscolombia,ministro NEAR ruiz,fakenews,hospital NEAR cancerol?gico,"""ministro de justicia"""
2,protestas NEAR colombia,ministro NEAR salud,falso,r?g?men* NEAR subsidiado,paraguay
3,protesta*,comisi?n NEAR s?ptima,fals*,r?g?men* NEAR especial,"""wilson ruíz"""
4,paro,trabajador* NEAR salud,mentira*,seguro NEAR m?dico,NaN
5,bloqueo*,paciente* NEAR enfermo,engaño*,r?g?men* NEAR excepcional,NaN
6,manifestaciones NEAR colombia,comisi?n NEAR s?ptima,mentir*,modelo NEAR usa,NaN
7,manifesta*,paciente* NEAR enfermo,NaN,salud NEAR usa,NaN
8,movilizaci?n,NaN,NaN,salud NEAR estados,NaN
9,moviliza* NEAR social,NaN,NaN,salud NEAR norteamerican*,NaN


In [10]:
q.get_logic_expression()

Paro AND Actores AND Reforma a la salud AND Instituciones AND Filtro por términos en tweets falsos AND Filtro por palabras de desinformación AND NOT


In [34]:
q.get_query()

'(((paro NEAR nacional) OR (soscolombia) OR (protestas NEAR colombia) OR (protesta*) OR (paro) OR (bloqueo*) OR (manifestaciones NEAR colombia) OR (manifesta*) OR (movilizaci?n) OR (moviliza* NEAR social) OR (estallido NEAR social) OR (sos NEAR colombia) OR (tributaria NEAR salud)) AND ("fernando ruiz" OR (ministro NEAR ruiz) OR (ministro NEAR salud) OR (comisi?n NEAR s?ptima) OR (trabajador* NEAR salud) OR (paciente* NEAR enfermo) OR (comisi?n NEAR s?ptima) OR (paciente* NEAR enfermo) OR (salud NEAR p?blica) OR (salud NEAR derecho*) OR (salud NEAR gratuita) OR (atenci?n NEAR salud) OR (reforma NEAR salud) OR (privatiza* NEAR salud) OR (SGSSS) OR "Sistema General de Seguridad Social en Salud" OR (sistema NEAR salud) OR (cartera NEAR salud) OR (ley NEAR 010) OR (ley NEAR salud) OR "proyecto de ley 010" OR (asegurador* NEAR intermediarias) OR (asegurador* NEAR salud) OR (salud NEAR usa) OR (salud NEAR estados) OR (salud NEAR norteamerican*) OR (salud NEAR gring?) OR (salud NEAR municip*)